# 基于4433法则和高被选原则的基金选取

Author：lusong@lsec.cc.ac.cn
Date:2021.2.7

### 导入所需要的包

In [1]:
# encoding=utf-8
import pandas as pd
import requests
from lxml import etree
import re
import collections

### 获取基金数据
ft 表示基金的类型，包含全部（all）、股票型（gp）、混合型（hh）、债券型（zq）、指数型（zs）、QDII（qdii）、LOF（lof）、FOF（fof）。

sc 表示排序类别，包括代码（dm）、简称（jc）、截止日期（jzrq）、单位净值（dwjz）、累计净值（ljjz）、日增长率（rzdf）、近一周（zzf）、近一月（1yzf）、近三月（3yzf）、近六月（6yzf）、近一年（1nzf）、近二年（2nzf）、近三年（3nzf）、今年来（jnzf）、成立以来（lnzf）、自定义时间内（qjzf）。

st 表示排序方式，包括降序（desc）和升序（asc）。

dx 表示是否可购，包括可购买（1）和不可购（0）。

In [2]:
header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36',
        'Referer': 'http://fund.eastmoney.com/data/fundranking.html',
        'Cookie':'st_si=74949607860286; st_asi=delete; ASP.NET_SessionId=gekyucnll0wte0wrks2rr23b; _adsame_fullscreen_18503=1; EMFUND1=null; EMFUND2=null; EMFUND3=null; EMFUND4=null; EMFUND5=null; EMFUND6=null; EMFUND7=null; EMFUND8=null; EMFUND0=null; EMFUND9=02-07 16:37:21@#$%u521B%u91D1%u5408%u4FE1%u5DE5%u4E1A%u5468%u671F%u80A1%u7968A@%23%24005968; st_pvi=90009717841707; st_sp=2021-02-07%2012%3A14%3A29; st_inirUrl=https%3A%2F%2Fwww.baidu.com%2Flink; st_sn=21; st_psi=2021020716562364-0-0372414431'
}
sample = '1500';
ft = 'gp'
sc = '6yzf'
dx = '1'
st = 'desc'
url = 'http://fund.eastmoney.com/data/rankhandler.aspx?op=ph&dt=kf&ft='+ft+'&rs=&gs=0&sc='+sc+'&st='+st+'&sd=2021-01-07&ed=2021-02-07&qdii=&tabSubtype=,,,,,&pi=1&pn='+sample+'&dx='+dx+'&v=0.2692835962833908'
response = requests.get(url=url, headers=header)
text = response.text

### 基金原数据预处理

In [3]:
data = text.split('=')[1]#使用等号分开去后面一部分
compile_data = re.findall("{datas:\\[(.*)\\],allRecords", str(data))[0]#取datas和allRecords中的所有内容
strip_data = str(compile_data).strip('[').strip(']')#移除字符串头尾的中括号
replace_quta = strip_data.replace('"', "")#双引号替换为空
quota_arrays = replace_quta.split(",")#使用逗号转为列表

### 整理或者基金数据表

In [4]:
intervals = [[i * 25, (i + 1) * 25] for i in range(15000)]#生成10000个区间，每个区间长度为25
narrays = []
for k in intervals:
    start, end = k[0], k[1]
    line = quota_arrays[start:end]#将条目25个分为一组,表示一只基金
    narrays.append(line)
header = ["基金代码", "基金简称", "基金条码", "日期",
          "单位净值", "累计净值", "日增长率", "近1周", "近1月", "近3月", "近半年", "近1年", "近2年", "近3年",
          "今年来", "成立来", "其他1", "其他2", "其他3", "其他4", "其他5", "其他6", "其他7", "其他8", "其他9"]
df = pd.DataFrame(narrays, columns=header)#生成pd数据结构
df.dropna()
total = df.count()[0]
print("共有{}支基金！".format(total))
df = df.head(total)
df_part = df[["基金代码", "基金简称", "日增长率", "近1周", "近1月", "近3月", "近半年", "近1年", "近2年", "近3年",
          "今年来", "成立来"]]#挑选部分感兴趣的条目
df.to_csv("./选取的全部基金数据.csv", encoding="utf_8_sig")

共有1263支基金！


### 使用4433原则挑选基金
4433原则，第一个4，近一年前1/4，第二个4，近两年，近三年，今年来前1/4，第一个3，近3个月前1/3，第二个3，近6个月前1/3

In [5]:
# 4433原则，第一个4，近一年前1/4，第二个4，近两年，近三年，今年来前1/4，第一个3，近3个月前1/3，第二个3，近6个月前1/3
total_quarter = int(total*0.25)
total_third = int(total*1/3)
df_picked_part = df_part
for sc in ["近1年","近2年","近3年","今年来"]:
    #print(sc)
    df_tmp = df_part.sort_values(by=[sc], ascending=False, axis=0)
    df_tmp = df_tmp.head(total_quarter)
    df_picked_part = pd.merge(df_picked_part,df_tmp,how='inner')
for sc in ["近3月","近半年"]:
    #print(sc)
    df_tmp = df_part.sort_values(by=[sc], ascending=False, axis=0)
    df_tmp = df_tmp.head(total_third)
    df_picked_part = pd.merge(df_picked_part,df_tmp,how='inner')

In [6]:
print(df_picked_part)
df_picked_part.to_csv("./4433法则挑选出的基金.csv", encoding="utf_8_sig")

Empty DataFrame
Columns: [基金代码, 基金简称, 日增长率, 近1周, 近1月, 近3月, 近半年, 近1年, 近2年, 近3年, 今年来, 成立来]
Index: []


### 获得各个基金的前几持仓数据和股票集合

In [7]:
rank_codes = df_part['基金代码'].values.tolist()
#len_codes = len(rank_codes)
stocks_array = []
stock_funds = []
total_part = int(total/100) #每百分之一报一次进度
for index, code in enumerate(rank_codes):
#    if index < 1:
#        print("<" * 30 + "所有基金的股票池前10情况" + ">" * 30)
#    print(code)
    if index%total_part==0:
        print("<" * 30 + "获得各个基金的前几持仓数据和股票集合中："+str(index)+"/"+str(total)+ ">" * 30)
    url = "http://fundf10.eastmoney.com/FundArchivesDatas.aspx?type=jjcc&code={}&topline=10&year=&month=&rt=0.5032668912422176".format(code)
    head = {
    "Cookie": "EMFUND1=null; EMFUND2=null; EMFUND3=null; EMFUND4=null; EMFUND5=null; EMFUND6=null; EMFUND7=null; EMFUND8=null; EMFUND0=null; st_si=44023331838789; st_asi=delete; EMFUND9=08-16 22:04:25@#$%u4E07%u5BB6%u65B0%u5229%u7075%u6D3B%u914D%u7F6E%u6DF7%u5408@%23%24519191; ASP.NET_SessionId=45qdofapdlm1hlgxapxuxhe1; st_pvi=87492384111747; st_sp=2020-08-16%2000%3A05%3A17; st_inirUrl=http%3A%2F%2Ffund.eastmoney.com%2Fdata%2Ffundranking.html; st_sn=12; st_psi=2020081622103685-0-6169905557"
    ,
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36"}
    response = requests.get(url, headers=head)
    text = response.text
    div = re.findall('content:\\"(.*)\\",arryear', text)[0]
    html_body = '<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>test</title></head><body>%s</body></html>' % (
        div)#构造网页
    html = etree.HTML(html_body)#将传进去的字符串转变成_Element对象
    stock_info = html.xpath('//div[1]/div/table/tbody/tr/td/a')
  #  stock_money = html.xpath('//div[1]/div/table/tbody/tr/td')
    stock_one_fund = []
    for i in range(0,len(stock_info)):
        stock = stock_info[i]
        tmp = stock.text.split('.')[0]
        if stock.text and (tmp.isdigit() or (tmp.isupper() and tmp.isalnum())):
#        if stock.text and stock.text.isdigit():
           # list_tmp = [stock.text,stock_info[i+1].text]
            stock_one_fund.append(stock_info[i+1].text)
#            print(stock_info[i+1].text)
    if len(stock_one_fund)>1:
  #      print("基金代码：{}".format(code), "基金持有前10股票池", stock_one_fund)
        stock_funds.append([code,stock_one_fund])
    if len(stock_one_fund) > 1 and stock_one_fund:
        stocks_array.extend(stock_one_fund)
print("<" * 30 + "获得各个基金的前几持仓数据和股票集合中：done!!!"+ ">" * 30)
pd.DataFrame(stock_funds).to_csv("./基金的股票持仓情况统计.csv", encoding="utf_8_sig")

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<获得各个基金的前几持仓数据和股票集合中：0/1263>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<获得各个基金的前几持仓数据和股票集合中：12/1263>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<获得各个基金的前几持仓数据和股票集合中：24/1263>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<获得各个基金的前几持仓数据和股票集合中：36/1263>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<获得各个基金的前几持仓数据和股票集合中：48/1263>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<获得各个基金的前几持仓数据和股票集合中：60/1263>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<获得各个基金的前几持仓数据和股票集合中：72/1263>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<获得各个基金的前几持仓数据和股票集合中：84/1263>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


KeyboardInterrupt: 

In [8]:
len(stock_info)

60

In [11]:
for i in stock_info:   
    print(i.text)

600809
山西汾酒
变动详情
股吧
行情
档案
300015
爱尔眼科
变动详情
股吧
行情
档案
000858
五粮液
变动详情
股吧
行情
档案
000596
古井贡酒
变动详情
股吧
行情
档案
600519
贵州茅台
变动详情
股吧
行情
档案
300558
贝达药业
变动详情
股吧
行情
档案
000661
长春高新
变动详情
股吧
行情
档案
000568
泸州老窖
变动详情
股吧
行情
档案
600600
青岛啤酒
变动详情
股吧
行情
档案
002410
广联达
变动详情
股吧
行情
档案


### 统计最受欢迎的n大股票

In [ ]:
rank = 10
count_each_stock = collections.Counter(stocks_array) #各个股票计数
df_stock=pd.DataFrame.from_dict(count_each_stock,orient='index',columns=["持有该股基金数目"])
df_stock=df_stock.reset_index().rename(columns={"index":"股票简称"})
df_stock=df_stock.sort_values(by="持有该股基金数目",ascending=False)
df_stock_head = df_stock.head(rank)
print(df_stock_head)
df_stock.to_csv("./股票的归属基金数目统计.csv", encoding="utf_8_sig")

### 计算出持仓最吻合最受欢迎股票的基金

In [ ]:
funds_stocks_count = []
for st_funds in stock_funds:
    df_stock_funds = pd.DataFrame(st_funds[1],columns=['股票简称'])
#    print(df_stock_funds)
    count = pd.merge(df_stock_head['股票简称'],df_stock_funds,how='inner').iloc[:,0].size
    jc_tmp = df_part[df_part['基金代码']==st_funds[0]].iloc[0,1]
    funds_stocks_count.append([jc_tmp,count])
df_funds_stock_count = pd.DataFrame(funds_stocks_count,columns = ['基金简称','优仓数目'])
df_funds_stock_count = df_funds_stock_count.sort_values(by=["优仓数目"], ascending=False, axis=0)
df_funds_stock_count = pd.merge(df_funds_stock_count,df_part,how='inner',on='基金简称')
df_funds_stock_count.to_csv("./基金持有优仓数目统计.csv", encoding="utf_8_sig")
df_funds_stock_count.head(10)